In [1]:
import pandas as pd

Step 1 :-  Clean the csv file and remove unnecessary data


In [2]:
def clean_csv(input_file, output_file, columns_to_remove, columns_to_check_for_missing):
    df = pd.read_csv(input_file)
    df.drop(columns=columns_to_remove, inplace=True)
    df.dropna(subset=columns_to_check_for_missing, inplace=True)
    df.to_csv(output_file, index=False)

In [23]:
input_file = 'PremierLeague.csv'
output_file = 'cleanPremierLeague.csv'

columns_to_remove = ['MatchID', 'MatchWeek', 'Date', 'Time', 'FullTimeHomeTeamGoals', 'FullTimeAwayTeamGoals', 'HalfTimeHomeTeamGoals', 'HalfTimeAwayTeamGoals', 'HalfTimeResult', 'Referee', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsOnTarget', 'AwayTeamShotsOnTarget', 'HomeTeamCorners', 'AwayTeamCorners', 'HomeTeamFouls', 'AwayTeamFouls', 'HomeTeamYellowCards', 'AwayTeamYellowCards', 'MarketMaxHomeTeam', 'MarketMaxDraw', 'MarketMaxAwayTeam', 'MarketAvgHomeTeam', 'MarketAvgDraw', 'MarketAvgAwayTeam', 'MarketMaxOver2.5Goals', 'MarketAvgOver2.5Goals', 'MarketAvgUnder2.5Goals', 'MarketMaxUnder2.5Goals', 'HomeTeamPoints', 'AwayTeamPoints']
columns_to_check_for_missing = ['FullTimeResult', 'B365HomeTeam', 'B365AwayTeam']

# Clean the CSV file
clean_csv(input_file, output_file, columns_to_remove, columns_to_check_for_missing)

Step 2 :- Finding bookies favourites according to odds

In [24]:
def findFavourites(file):
    df = pd.read_csv(file)
    df['Favourites'] = df.apply(lambda row: 'Home' if row['B365HomeTeam'] < row['B365AwayTeam'] else 'Away', axis = 1)
    df.to_csv(file, index=False)


In [25]:
findFavourites(output_file)

step 3 :- Calculating profits if we only bet on favorites

In [26]:
def calculateProfitsOnFavourites(file):
    df = pd.read_csv(file)

    def calculateProfit(row):
        betAmount = 100
        if row['Favourites'] == 'Home':
            if row['FullTimeResult'] == 'H':
                return betAmount * row['B365HomeTeam'] - betAmount
            else:
                return -betAmount
            
        elif row['Favourites'] == 'Away':
            if row['FullTimeResult'] == 'A':
                return betAmount * row['B365AwayTeam'] - betAmount
            else:
                return -betAmount
            
    df['ROIonFavourites'] = df.apply(calculateProfit, axis=1)
    df.to_csv(file, index=False)

calculateProfitsOnFavourites(output_file)

In [27]:

df=pd.read_csv(output_file)
print(df)

         Season     HomeTeam       AwayTeam FullTimeResult  HomeTeamRedCards  \
0     2002-2003    Blackburn     Sunderland              D               0.0   
1     2002-2003     Charlton        Chelsea              A               1.0   
2     2002-2003      Everton      Tottenham              D               0.0   
3     2002-2003       Fulham         Bolton              H               0.0   
4     2002-2003        Leeds       Man City              H               0.0   
...         ...          ...            ...            ...               ...   
8425  2024-2025     West Ham        Ipswich              H               0.0   
8426  2024-2025      Everton      Newcastle              D               0.0   
8427  2024-2025  Aston Villa     Man United              D               0.0   
8428  2024-2025      Chelsea  Nott'm Forest              D               0.0   
8429  2024-2025     Brighton      Tottenham              H               0.0   

      AwayTeamRedCards  B365HomeTeam  B

In [28]:
totalAmountBet = len(df)*100
netProfit = df['ROIonFavourites'].sum()
roi = (netProfit/totalAmountBet) * 100

print("Your ROI if you always placed bets on the bookies favourite.")

print("Total Amount Bet: ", totalAmountBet)
print("Net Profit: ", netProfit)
print(f"Return On Investment: {roi:.2f}%")

Your ROI if you always placed bets on the bookies favourite.
Total Amount Bet:  843000
Net Profit:  -26164.899999999998
Return On Investment: -3.10%


Step 4:- Calculating ROI on Underdogs

In [29]:
def calculateProfitsOnUnderdogs(file):
    df = pd.read_csv(file)

    def calculateProfit(row):
        betAmount = 100
        if row['Favourites'] == 'Home':
            if row['FullTimeResult'] == 'A':
                return betAmount * row['B365AwayTeam'] - betAmount
            else:
                return -betAmount
            
        elif row['Favourites'] == 'Away':
            if row['FullTimeResult'] == 'H':
                return betAmount * row['B365HomeTeam'] - betAmount
            else:
                return -betAmount
            
    df['ROIonUnderdogs'] = df.apply(calculateProfit, axis=1)
    df.to_csv(file, index=False)

calculateProfitsOnUnderdogs(output_file)

In [32]:
df = pd.read_csv(output_file)

totalAmountBet = len(df)*100
netProfit = df['ROIonUnderdogs'].sum()
roi = (netProfit/totalAmountBet) * 100

print("Your ROI if you always placed bets on the underdogs.")

print("Total Amount Bet: ", totalAmountBet)
print("Net Profit: ", netProfit)
print(f"Return On Investment: {roi:.2f}%")

Your ROI if you always placed bets on the underdogs.
Total Amount Bet:  843000
Net Profit:  -80084.9
Return On Investment: -9.50%


Step 5 : Calculating ROI when only betting Odds

In [33]:
def calculateProfitsOnDraws(file):
    df = pd.read_csv(file)

    def calculateProfit(row):
        betAmount = 100
        if row['FullTimeResult'] == 'D':
            return betAmount * row['B365Draw'] - betAmount
        else:
            return -betAmount
            
    df['ROIonDraws'] = df.apply(calculateProfit, axis=1)
    df.to_csv(file, index=False)

calculateProfitsOnDraws(output_file)

In [35]:
df = pd.read_csv(output_file)

totalAmountBet = len(df)*100
netProfit = df['ROIonDraws'].sum()
roi = (netProfit/totalAmountBet) * 100

print("Your ROI if you always placed bets on noone wining.")

print("Total Amount Bet: ", totalAmountBet)
print("Net Profit: ", netProfit)
print(f"Return On Investment: {roi:.2f}%")

Your ROI if you always placed bets on noone wining.
Total Amount Bet:  843000
Net Profit:  -65889.5
Return On Investment: -7.82%


As Expected the safer bets i.e betting on Favourites was safer but that still resulted in overall loss of money :(. Lets see what happens if we bet on even safer bets, i.e betting on double chance bets where you select two of these outcomes, meaning you win the bet if either of your chosen outcomes occurs. The formula for calculating odds for double chance is (1/(probability of outcome 1 + probability of outcome 2)).

Step 6:- Calculating FavouriteX (Favourites to either win or draw) odds and then calculating its ROI.

In [37]:
def calculateFavoriteXOdds(file):
    df = pd.read_csv(file)

    def calculateDoubleChanceOdds(row):
        if row['Favourites'] == 'Home':
            oddsFav = row['B365HomeTeam']
        elif row['Favourites'] == 'Away':
            oddsFav = row['B365AwayTeam']
        else:
            return None
        
        oddsDraw = row['B365Draw']

        doubleChanceOdds = 1 / ((1/oddsFav) + (1/oddsDraw))
        return doubleChanceOdds
    
    df['FavouriteXOdds'] = df.apply(calculateDoubleChanceOdds, axis=1)
    df.to_csv(file, index=False)

calculateFavoriteXOdds(output_file)

In [40]:
def calculateProfitsOnFavouriteX(file):
    df = pd.read_csv(file)

    def calculateProfit(row):
        betAmount = 100
        if row['Favourites'] == 'Home':
            if (row['FullTimeResult'] == 'H') or (row['FullTimeResult'] == 'D'):
                return betAmount * row['FavouriteXOdds'] - betAmount
            else:
                return -betAmount
            
        elif row['Favourites'] == 'Away' :
            if row['FullTimeResult'] == 'A' or row['FullTimeResult'] == 'D':
                return betAmount * row['FavouriteXOdds'] - betAmount
            else:
                return -betAmount
            
    df['ROIonFavouriteX'] = df.apply(calculateProfit, axis=1)
    df.to_csv(file, index=False)

calculateProfitsOnFavouriteX(output_file)

In [41]:
df = pd.read_csv(output_file)

totalAmountBet = len(df)*100
netProfit = df['ROIonFavouriteX'].sum()
roi = (netProfit/totalAmountBet) * 100

print("Your ROI if you always placed bets on the Favourite winning or atleast drawing.")

print("Total Amount Bet: ", totalAmountBet)
print("Net Profit: ", netProfit)
print(f"Return On Investment: {roi:.2f}%")

Your ROI if you always placed bets on the Favourite winning or atleast drawing.
Total Amount Bet:  843000
Net Profit:  -34396.39985201236
Return On Investment: -4.08%
